In [40]:
import os
import pandas as pd

In [41]:
MOVEMENTS = (
    "elbflex",
    "shoabd",
    "shoext",
    "shoflex",
    "hipabd",
    "hipext",
    "hipflex",
    "kneeflex",
)
GROUPS = (
    "left",
    "right"
)

In [42]:
# Get path to participant & movement model generated CSVs.
# 
# trial_info dictionary:
# {
#   "dl200": {
#              "left_upper_elbflex": {"total_frames": 346, "good_frames": 70, "drop_percent": 0.80},
#              "left_upper_shoabd":  {...},
#              ...
#            },
#   "dl201": { ... },
#   ...
# }

trial_info = dict()
for movement in MOVEMENTS:
    path = f"../gen/{movement}/model/"
    for filename in os.listdir(path):
        if filename.endswith('.csv'):
            participant = filename.split('_')[0] # e.g. dl200
            group = [g for g in GROUPS if g in filename][0]
            movement = [m for m in MOVEMENTS if m in filename][0]
            group_and_movement = f"{group}_{movement}"
            trial_num = filename.strip('.csv').split('_')[-1]
            trial_df = pd.read_csv(f"{path}/{filename}", index_col=0)
            joint_names = [col for col in trial_df.columns if movement[:3] in col]
            if not joint_names:
                print(f"Skipping trial {filename} due to no {movement[:3]} columns.")
                continue
            joint_name = joint_names[0]
            total_frames = trial_df.shape[0]
            good_frames = trial_df[joint_name].dropna().shape[0]
            drop_percent = round((total_frames - good_frames) / total_frames, 2)
            
            trial_info.setdefault(participant, {})
            trial_info[participant].setdefault(group_and_movement, {})
            trial_info[participant][group_and_movement][trial_num] = {
                "total_frames": total_frames,
                "good_frames": good_frames,
                "drop_percent": drop_percent
            }

Skipping trial dl214_left_uppper_elbflex_006.csv due to no elb columns.
Skipping trial dl214_left_uppper_shoabd_017.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_001.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_009.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_006.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_019.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_007.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_013.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_015.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_016.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_004.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_018.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_005.csv due to no sho columns.
Skipping trial dl214_left_uppper_shoabd_014.csv due to no sho columns.
Skipp

In [43]:
trial_info

{'dl211b': {'left_elbflex': {'001': {'total_frames': 150,
    'good_frames': 20,
    'drop_percent': 0.87},
   '019': {'total_frames': 164, 'good_frames': 29, 'drop_percent': 0.82},
   '018': {'total_frames': 183, 'good_frames': 43, 'drop_percent': 0.77},
   '008': {'total_frames': 172, 'good_frames': 53, 'drop_percent': 0.69},
   '011': {'total_frames': 173, 'good_frames': 47, 'drop_percent': 0.73},
   '004': {'total_frames': 141, 'good_frames': 29, 'drop_percent': 0.79},
   '009': {'total_frames': 176, 'good_frames': 50, 'drop_percent': 0.72},
   '003': {'total_frames': 144, 'good_frames': 7, 'drop_percent': 0.95},
   '002': {'total_frames': 158, 'good_frames': 21, 'drop_percent': 0.87},
   '020': {'total_frames': 183, 'good_frames': 64, 'drop_percent': 0.65},
   '007': {'total_frames': 163, 'good_frames': 62, 'drop_percent': 0.62},
   '013': {'total_frames': 189, 'good_frames': 56, 'drop_percent': 0.7},
   '005': {'total_frames': 153, 'good_frames': 49, 'drop_percent': 0.68},
   '00

In [44]:
summary_dict = {}
for participant, gm_dict in trial_info.items():
    for group_and_movement, trial_dict in gm_dict.items():
        drop_percent_avg = 0
        for trial_num, info_dict in trial_dict.items():
            drop_percent_avg += info_dict['drop_percent']
        drop_percent_avg /= len(trial_dict)
        summary_dict.setdefault(participant, {})
        summary_dict[participant].setdefault(group_and_movement, {})
        summary_dict[participant][group_and_movement] = round(drop_percent_avg, 2)

In [45]:
pd.set_option('display.max_rows', None)
summary_df = pd.DataFrame(summary_dict)
summary_df = summary_df.reindex(sorted(summary_df.columns), axis=1)
summary_df

,dl200,dl201,dl202,dl203,dl204,dl205,dl209,dl210,dl210a,dl211b,dl212,dl213,dl214
left_elbflex,0.34,0.22,0.26,0.70,0.26,0.02,NaN,NaN,NaN,0.74,NaN,0.89,0.96
right_elbflex,0.70,0.21,0.38,0.22,0.27,0.00,NaN,NaN,0.01,0.49,0.53,0.42,0.02
left_shoabd,0.30,0.28,0.25,0.71,0.07,0.27,NaN,NaN,NaN,0.09,0.89,0.13,0.97
right_shoabd,0.44,0.30,0.09,0.22,0.30,0.28,NaN,NaN,0.35,0.20,0.11,0.47,0.40
left_shoext,0.32,0.16,0.64,0.31,0.33,0.03,0.36,0.91,NaN,0.38,0.75,0.86,0.49
right_shoext,0.79,0.31,0.51,0.24,0.02,0.01,NaN,0.15,0.00,0.26,0.63,0.27,0.00
right_shoflex,0.78,0.35,0.58,0.10,0.13,0.04,NaN,0.33,0.01,0.48,0.35,0.14,0.13
left_shoflex,0.38,0.43,0.57,0.57,0.75,0.03,NaN,0.89,NaN,0.60,0.60,0.73,0.14
